In [1]:
import win32com.client as win32
import pyspark.pandas as pd
import numpy as np

from openpyxl import load_workbook
from openpyxl.pivot.fields import Missing

import os

In [2]:
#Conversão do arquivo xls para xlsx

absPath = os.path.abspath('datalake/vendas-combustiveis-m3.xls')
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(absPath)

wb.SaveAs(absPath+'x', FileFormat = 51)
wb.Close()                         
excel.Application.Quit()

In [3]:
#Função para extração da Pivot cached table

def extract_pivot_cached(pivot_name, report_name):

    pivot_table = [p for p in worksheet._pivots if p.name == pivot_name][0]

    fields_map = {}
    for field in pivot_table.cache.cacheFields:
        if field.sharedItems.count > 0:
            fields_map[field.name] = [f.v for f in field.sharedItems._fields]

    column_names = [field.name for field in pivot_table.cache.cacheFields]
    rows = []
    for record in pivot_table.cache.records.r:
        record_values = [
            field.v if not isinstance(field, Missing) else np.nan for field in record._fields
        ]

        row_dict = {k: v for k, v in zip(column_names, record_values)}

        for key in fields_map:
            row_dict[key] = fields_map[key][row_dict[key]]

        rows.append(row_dict)

    df_pivot = pd.DataFrame.from_dict(rows)
    df_pivot.to_parquet('datalake/bronze/'+report_name)  

In [4]:
# Leitura do arquivo e extração das pivot cached tables
workbook = load_workbook('datalake/vendas-combustiveis-m3.xlsx')
worksheet = workbook['Plan1']

report_name = 'oil'
pivot_name ='Tabela dinâmica1'
extract_pivot_cached(pivot_name, report_name)

report_name = 'diesel'
pivot_name ='Tabela dinâmica3'
extract_pivot_cached(pivot_name, report_name)

C:\Spark\spark-3.3.0-bin-hadoop2\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_parquet`, the existing index is lost when converting to Parquet.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
C:\Spark\spark-3.3.0-bin-hadoop2\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
C:\Spark\spark-3.3.0-bin-hadoop2\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_parquet`, the existing index is lost when converting to Parquet.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
C:\Spark\spark-3.3.0-bin-hadoop2\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
